In [ ]:
from __future__ import print_function, absolute_import, division

import logging
import os
import sys
from collections import defaultdict, OrderedDict
from errno import ENOENT, ENODATA
from stat import S_IFDIR, S_IFLNK, S_IFREG
from sys import argv, exit, path
from time import time

from fuse import FUSE, FuseOSError, Operations, LoggingMixIn

# zzz 
sys.path.append("../tagfuse/tagfuse")
print(os.getcwd())
print(path)
from Si446xUtils import si446x_device_enable, path2tlvs
from taghandlers import *
from TagFuseTree import TagFuseFileTree

if not hasattr(__builtins__, 'bytes'):
    bytes = str

In [ ]:
radio = si446x_device_enable()

In [ ]:
# default paramters
MAX_WAIT            = 10
MAX_RECV            = 255
MAX_PAYLOAD         = 254
MAX_RETRIES         = 10
RADIO_POWER         = 100
SHORT_DELAY         = 0
from datetime import datetime

In [ ]:
def path2list(path):
    path = os.path.abspath(os.path.realpath(path))
    return path.split('/')[1:]

def LocateNode(tag_tree, path):
    if (path == '/'):
        print('located root')
        return tag_tree
    return tag_tree.traverse(path2list(path), 0)

def DeleteNode(path, node):
    pass

In [ ]:
tag_tree = PollNetDirHandler(radio, OrderedDict([
            ('',                       FileHandler(S_IFDIR, 0o751, 3)),
            ('<node_id:0xffffffffffff>', TagFuseFileTree(radio)),
            ]))

In [ ]:
print(tag_tree)

In [ ]:
path='/<node_id:0xffffffffffff>/tag/sd/0/dblk/byte/0'
handler = LocateNode(tag_tree, path)
print(handler)

In [ ]:
handler.getattr(path2list(path), update=True)

In [ ]:
path='/<node_id:0xffffffffffff>/tag/sd/0/panic/byte/0'
handler = LocateNode(tag_tree, path)
handler.getattr(path2list(path), update=True)

In [ ]:
path2tlvs(['<gps:(1,2,3)>'])

In [ ]:
STOP

In [ ]:
BLOCK_SIZE = 512
def verify_file(radio, check_fname, dev, unit, skip=0):
    with open(check_fname, 'rb') as fd:
        start  = datetime.now()
        eof    = False
        offset  = 0
        while not eof:
            fd.seek(offset+skip)
            dbuf = bytearray(fd.read(BLOCK_SIZE))
            sbuf, eof = file_get_bytes(radio, dev, unit, BLOCK_SIZE, offset)
            if sbuf:
                for ix in range(len(sbuf)):
                    if sbuf[ix] != dbuf[ix]:
                        print(hexlify(sbuf))
                        print(hexlify(dbuf))
                        print("\r{} mismatch: offset:{}, {}/{}".format(datetime.now() - start,
                                                            offset+ix,
                                                            hexlify(sbuf[ix-4:ix+4]),
                                                            hexlify(dbuf[ix-4:ix+4])))
                        break
                offset += len(sbuf)
            elif not eof:
                print("\r{} {}".format(datetime.now() - start, 'timeout'), end="")
            sleep(0)
            print("\r{} {}".format(datetime.now() - start, offset), end="")
    print("\ntime: {}, offset: {}".format(datetime.now() - start, offset))

In [ ]:
verify_file(radio, '/home/pi/o/mm/dblk.data', 'dblk', '0')

In [ ]:
verify_file(radio, '/home/pi/o/mm/panic.data', 'panic', '2', skip=512)

In [ ]:
dblk_put_note(radio, 'hello')

In [ ]:
file_update_attrs(radio, ['panic', 'byte', 3], {})

In [ ]:
file_update_attrs(radio, ['dblk', 'byte', '0'], {})

In [ ]:
file_update_attrs(radio, ['dblk', 'note'], {})